In [1]:
import pandas as pd
from ALLCools.mcds import MCDS
from scipy.stats import f_oneway, ttest_ind, kruskal
import joblib
import anndata
from wmb import mm10
import pingouin as pg
import xarray as xr
import joblib
import numpy as np
import glob

In [2]:
gene_name_to_id = joblib.load('240112_DMG_CEF/gene_name_to_id_dict')
gene_id_to_name = joblib.load('240112_DMG_CEF/gene_id_to_name_dict')

/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [3]:
_gender = 'Male'
mc_type = 'CHN'

In [4]:
if _gender == 'Female':
    mcds_paths = "Female.genes_frac.mcds"
    var_dim = "gene"
    meta = pd.read_csv('/home/qzeng/project/aging/metadata/240104_m3C_META.csv', index_col = 0)
    use_cts = joblib.load(f"/home/qzeng/project/aging/metadata/m3c_use_cts")
elif _gender == 'Male':
    mcds_paths = "Male.genes_frac.mcds"
    var_dim = "geneslop2k"
    meta = pd.read_csv('/home/qzeng/project/aging/metadata/240104_mC_META.csv', index_col = 0)
    use_cts = joblib.load(f"/home/qzeng/project/aging/metadata/mc_use_cts")

/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
use_region_cts = ['.'.join(fp.split('/')[-1].split('.')[:2]) for fp in 
                  glob.glob('/ceph/gale-1/qzeng/AmbData/AgingMajorType.Region.Age/*')]
use_region_cts = list(set(use_region_cts))
use_region_cts

['Oligo_NN.Caudate_Putamen',
 'Oligo_NN.Frontal_Cortex',
 'STR_D1_Gaba.Caudate_Putamen',
 'STR-PAL_Chst9_Gaba.Nucleus_accumbens',
 'Oligo_NN.Nucleus_accumbens',
 'Pvalb_Gaba.Entorhinal_Cortex',
 'LA-BLA-BMA-PA_Glut.Amygdala',
 'DG_Glut.Anterior_Hippocampus',
 'L23_IT_PPP_Glut.Posterior_Hippocampus',
 'STR-PAL_Chst9_Gaba.Amygdala',
 'CA3_Glut.Posterior_Hippocampus',
 'CEA-BST_Gaba.Amygdala',
 'STR_D2_Gaba.Amygdala',
 'Pvalb_Gaba.Frontal_Cortex',
 'STR_D2_Gaba.Caudate_Putamen',
 'Oligo_NN.Amygdala',
 'Sst_Gaba.Posterior_Hippocampus',
 'L23_IT_PPP_Glut.Entorhinal_Cortex',
 'STR_D1_Gaba.Nucleus_accumbens',
 'LA-BLA-BMA-PA_Glut.Entorhinal_Cortex',
 'Oligo_NN.Anterior_Hippocampus',
 'DG_Glut.Posterior_Hippocampus',
 'Oligo_NN.PAG-PCG',
 'Oligo_NN.Posterior_Hippocampus',
 'CA3_Glut.Anterior_Hippocampus',
 'STR_D1_Gaba.Amygdala',
 'CEA-BST_Gaba.Nucleus_accumbens',
 'CA1-ProS_Glut.Anterior_Hippocampus',
 'L6bCT_ENT_Glut.Posterior_Hippocampus',
 'CA1-ProS_Glut.Posterior_Hippocampus',
 'Sst_Gaba.

In [6]:
use_meta = meta[meta['AgingMajorType'].isin(use_cts)]
use_meta.loc[:,'AgingMajorType'] = use_meta['AgingMajorType'].apply(lambda x: x.replace('/','').replace(' ','_'))
use_meta.loc[:,'AgingMajorType.Region'] = use_meta['AgingMajorType'] + '.' + use_meta['Region']
use_meta = use_meta[use_meta['AgingMajorType.Region'].isin(use_region_cts)]

/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [7]:
mcds =  MCDS.open(mcds_paths, var_dim=var_dim, use_obs=use_meta.index).sel(mc_type=mc_type).load()

mcds.coords['AgingMajorType.Region'] = use_meta['AgingMajorType.Region']
mcds.coords['Age'] = use_meta['Age']
mcds

<xarray.MCDS>
Dimensions:                (cell: 65079, geneslop2k: 53088)
Coordinates:
  * cell                   (cell) object '3A_M_1453' ... 'AMB_210309_9mo_7H_8...
  * geneslop2k             (geneslop2k) <U21 'ENSMUSG00000102693.1' ... 'ENSM...
    geneslop2k_chrom       (geneslop2k) <U5 'chr1' 'chr1' ... 'chrX' 'chrX'
    geneslop2k_cov_mean    (geneslop2k) float64 53.12 42.42 ... 19.79 65.23
    geneslop2k_end         (geneslop2k) int64 3076321 3104124 ... 170680053
    geneslop2k_start       (geneslop2k) int64 3071252 3100015 ... 170670643
    mc_type                <U3 'CHN'
    strand_type            <U4 'both'
    AgingMajorType.Region  (cell) object 'Pvalb_Gaba.Frontal_Cortex' ... 'STR...
    Age                    (cell) object '8wk' '8wk' '8wk' ... '9mo' '9mo' '9mo'
Data variables:
    geneslop2k_da_frac     (cell, geneslop2k) float32 0.04045 ... 0.001577
Attributes:
    obs_dim:  cell
    var_dim:  geneslop2k

In [8]:
genes = mcds.get_index(var_dim)

In [9]:
df_list = []
region_celltypes = []

for mt_region ,tmp_mcds in mcds.groupby('AgingMajorType.Region'):

    tmp_meta = tmp_mcds['Age'].to_pandas()

    tmp_8wk_cells = tmp_meta[tmp_meta == '8wk'].index
    tmp_9mo_cells = tmp_meta[tmp_meta == '9mo'].index
    tmp_18mo_cells = tmp_meta[tmp_meta == '18mo'].index

    tmp_gene_frac = tmp_mcds[f"{var_dim}_da_frac"].to_pandas()

    F, p_value = f_oneway(
        tmp_gene_frac.loc[tmp_8wk_cells],
        tmp_gene_frac.loc[tmp_9mo_cells],
        tmp_gene_frac.loc[tmp_18mo_cells],
        axis=0,
    )
    celltype_gene_pvalue = pd.DataFrame({"pvalue": p_value}, index=genes)
    
    # fdr correction
    reject, pvals_corr = pg.multicomp(celltype_gene_pvalue[f'pvalue'], method="fdr_by")
    celltype_gene_pvalue['corrected_pvalue'] = pvals_corr

    tmp_gene_frac['Age'] = use_meta['Age']
    gene_mean_by_age = tmp_gene_frac.groupby('Age').mean().T
    gene_mean_by_age.columns = [f'{_age}' for _age in gene_mean_by_age.columns]

    celltype_gene_pvalue_and_mean = pd.concat([celltype_gene_pvalue, gene_mean_by_age], axis = 1)
    df_list.append(celltype_gene_pvalue_and_mean)
    region_celltypes.append(mt_region)
    print(mt_region)
    

CA1-ProS_Glut.Anterior_Hippocampus
CA1-ProS_Glut.Posterior_Hippocampus
CA3_Glut.Anterior_Hippocampus
CA3_Glut.Posterior_Hippocampus
CEA-BST_Gaba.Amygdala
CEA-BST_Gaba.Nucleus_accumbens
DG_Glut.Anterior_Hippocampus
DG_Glut.Posterior_Hippocampus
L23_IT_PPP_Glut.Entorhinal_Cortex
L23_IT_PPP_Glut.Posterior_Hippocampus
L6bCT_ENT_Glut.Entorhinal_Cortex
L6bCT_ENT_Glut.Posterior_Hippocampus
LA-BLA-BMA-PA_Glut.Amygdala
LA-BLA-BMA-PA_Glut.Entorhinal_Cortex
Oligo_NN.Amygdala
Oligo_NN.Anterior_Hippocampus
Oligo_NN.Caudate_Putamen
Oligo_NN.Frontal_Cortex
Oligo_NN.Nucleus_accumbens
Oligo_NN.PAG-PCG
Oligo_NN.Posterior_Hippocampus
Pvalb_Gaba.Entorhinal_Cortex
Pvalb_Gaba.Frontal_Cortex
STR-PAL_Chst9_Gaba.Amygdala
STR-PAL_Chst9_Gaba.Nucleus_accumbens
STR_D1_Gaba.Amygdala
STR_D1_Gaba.Caudate_Putamen
STR_D1_Gaba.Nucleus_accumbens
STR_D2_Gaba.Amygdala
STR_D2_Gaba.Caudate_Putamen
STR_D2_Gaba.Nucleus_accumbens
Sst_Gaba.Entorhinal_Cortex
Sst_Gaba.Posterior_Hippocampus


In [10]:
data_arrays = []
for region_ct, df in dict(zip(region_celltypes,df_list)).items():
    data = np.array([df])
    da = xr.DataArray(data, 
                      coords=[
                          [region_ct],
                          df.index, 
                          df.columns, 
                          ],
                      dims=['region_ct','genes', 'columns'])
    data_arrays.append(da)
total_data = xr.Dataset({f"anova_results": xr.concat(data_arrays, dim="region_ct")})
total_data

<xarray.Dataset>
Dimensions:        (region_ct: 33, genes: 53088, columns: 5)
Coordinates:
  * region_ct      (region_ct) <U37 'CA1-ProS_Glut.Anterior_Hippocampus' ... ...
  * genes          (genes) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000093...
  * columns        (columns) object 'pvalue' 'corrected_pvalue' ... '8wk' '9mo'
Data variables:
    anova_results  (region_ct, genes, columns) float64 8.183e-05 ... 0.01936

In [11]:
total_data.to_zarr(f'{mc_type}.{_gender}.AgingMajorType.Region.Anova.Result.zarr', mode='w')